In [184]:


import json
from openai import OpenAI
import os
import gradio as gr
import ollama
from dotenv import load_dotenv
#Model = "llama3.2"
#openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
  print(f"OpenAI API Key exists and begins {openai_api_key[:15]}")
else:
    print("OpenAI API Key not found")
MODEL = "gpt-4o-mini"
openai = OpenAI()


OpenAI API Key exists and begins sk-proj-JOqwkie


In [185]:
system_message = "You are a helpful assistant for an Airline called FlightAI."
system_message += "Give short, courteous answers, no more than 1 sentence."
system_message += "Always be accurate. if you don't know the answer, say so"

In [186]:
#The chat function
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

Running on local URL:  http://127.0.0.1:7904

To create a public link, set `share=True` in `launch()`.


In [187]:
gr.ChatInterface(fn=chat, type='messages').launch()

Running on local URL:  http://127.0.0.1:7905

To create a public link, set `share=True` in `launch()`.


In [188]:
#Tools: we write a function and have the LLM call that function as part of its response
#ticket_prices = {"lagos":"40k", "abuja":"150k", "benin": "70k", "uyo":"50k", "abeokuta":"25k"}
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city,"Unknown")


In [189]:
get_ticket_price("paris")

Tool get_ticket_price called for paris


'$899'

In [190]:
#The particular dictionary structure that's required to describe our function:
price_function = {
    "name":"get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters":{
        "type": "object",
        "properties":{
         "destination_city":{
            "type":"string",
            "description": "The city that the customer wants to travel to",
        },
    },
     "required":["destination_city"],
     "additionalProperties": False,
  }
}

In [191]:
tools = [{"type": "function", "function":price_function}]

In [192]:

#get the llm to call our function
def chat(message, history):
    messages =[{"role":"system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role":"assistant","content":assistant})
    messages.append({"role":"user", "content":message})
    #messages= [{"role":"system", "content":system_message}] + history + [{"role":"user", "content":message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason== 'tool_calls':
        message = response.choices[0].message
        response , city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content



In [193]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments =json.loads( tool_call.function.arguments)
    city = arguments.get("destination_city")
    price = get_ticket_price(city)
    response = {
        "role":"tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id":message.tool_calls[0].id

    }
    return response, city

In [194]:
gr.ChatInterface(
    fn=chat,
    title="Full Screen Chat",
    fill_height=True,
    css="""
        .gradio-container {max-width: 100% !important; height: 100vh !important; padding: 0 !important;}
        #component-0 {width: 100% !important; height: 100% !important; margin: 0 auto !important;}
        .wrap {height: 100% !important;}
    """
).launch()


Running on local URL:  http://127.0.0.1:7906

To create a public link, set `share=True` in `launch()`.
